In [6]:
!pip install featuretools

In [7]:
import featuretools as ft
import numpy as np
import pandas as pd
import os

In [8]:
from xgboost import XGBClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import pandas as pd
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE

from xgboost import plot_importance
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

import os
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [9]:
target = 'Segmentation'
key    = 'ID'

In [10]:
master = pd.read_csv('master_feat.csv')
master.shape, master[key].nunique()

((8068, 11), 8068)

In [11]:
master.set_index(key, inplace = True)

In [12]:
catVars = ['Gender','Ever_Married','Graduated','Profession','Spending_Score','Var_1']
numVars = ['Age','Work_Experience','Family_Size']

In [13]:
master[catVars] = master[catVars].astype(str)

### One-hot

In [14]:
# master.dtypes

In [15]:
categorical_features_indices = np.where((master.dtypes != 'int16') & (master.dtypes != 'int8') & (master.dtypes != 'int64') & (master.dtypes != 'float16') & (master.dtypes != 'float64'))[0]
catVars = list(set(master.iloc[:,categorical_features_indices].columns.tolist()) - set([key, target]))

In [16]:
one_hot = pd.get_dummies(master[catVars])

master = master.drop(catVars,axis = 1)
master = master.join(one_hot)

In [17]:
train = master[master[target] != '-1']
test  = master[master[target] == '-1']

In [18]:
del master
import gc
gc.collect()

0

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
train1, val1 = train_test_split(train, test_size=0.2, random_state=7)

In [21]:
train1.shape, val1.shape

((6454, 33), (1614, 33))

In [22]:
X_train = train1.drop(target, axis =1)
X_val   = val1.drop(target, axis =1)
X_test  = test.drop(target, axis =1)

y_train = train1[target]
y_val = val1[target]
y_test = test[target]

In [ ]:
###Hypertune the model intensively with a 5 fold cross validation strategy.
##first grid search for max_depth,min_child_weight then fix those and search for rest in same fashion
### parameter scale_pos_weight is quite important in case of imbalanced dataset

from sklearn.model_selection import GridSearchCV
param_test1 = {
  # 'min_child_weight':[1,50,100],
  'max_depth': [5,6,7],
  'n_estimators':[150,200,300,400],
  # 'scale_pos_weight':[1,2,3,4],
  'colsample_bytree':[0.7,0.8], 
  'subsample':[0.7,0.8]
}

gsearch1 = GridSearchCV(estimator = XGBClassifier(tree_method='gpu_hist',predictor='gpu_predictor', learning_rate=0.1, n_estimators=1000, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, nthread=-1, scale_pos_weight=3,seed=27), 
 param_grid = param_test1, scoring=['accuracy'],n_jobs=-1,iid=False, cv=5,refit='accuracy', verbose =100)

gsearch1.fit(X_train, y_train)
# print(gsearch1.grid_scores_)
print("gsearch1.best_params_",gsearch1.best_params_)
print("gsearch1.best_score_",gsearch1.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   17.7s
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed:   33.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:   33.1s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   40.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   40.8s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:   51.0s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:   51.1s
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  1

In [ ]:
%%time

modelXg = XGBClassifier(tree_method='gpu_hist',predictor='gpu_predictor',\
                        learning_rate=0.1, n_estimators=100, max_depth=5,min_child_weight=100,
                        nthread=-1, subsample=0.8, colsample_bytree=0.7, scoring = 'accuracy',seed=7)
modelXg.fit(X_train, y_train)
# scale_pos_weight=3

In [ ]:
feature_important  = modelXg.get_booster().get_score(importance_type='gain')

keys = list(feature_important.keys())
values = list(feature_important.values())

data = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score")
# data.plot(kind='barh')
data.sort_values('score', ascending = False)

In [ ]:
# predictions AUC on train and validation sets
print("Train Acc:", accuracy_score(y_train, modelXg.predict(X_train)))
print("Val Acc:",   accuracy_score(y_val,   modelXg.predict(X_val)))

In [ ]:
pd.DataFrame(modelXg.predict_proba(X_val)).to_csv('check.csv', index =False)

### Pred on test

In [ ]:
#### Prediction on test
output = "XGB_allFeat.csv"
test[target] = modelXg.predict(X_test)
test = test.reset_index()
# test[['Patient_ID','Health_Camp_ID']] = test[Key].str.split('|', 1, expand=True)
test[[key,target]].to_csv(output,index=False)

In [ ]:
from google.colab import files
files.download(output)

In [ ]:
test.Crop_Damage.value_counts(1)

In [ ]:
train.Crop_Damage.value_counts(1)